In [9]:
import re
import json
import logging
import time
import boto3

from konlpy.tag import Twitter
from hashlib import sha224

from elasticsearch import Elasticsearch
from elasticsearch import helpers
from redis import Redis

import requests
from bs4 import BeautifulSoup

import pandas as pd

In [18]:
logger = logging.getLogger(__name__) # "__name__"를 하면, root log는 제외시키고, 이 모듈 내에서 발생한 로그만 포함시킴. 없으면 requests 모듈에서 생긴 로그, boto3에서 생긴 로그 등이 포함되어 버림 그래서 지저분해짐
logger.setLevel(logging.INFO)
formatter = logging.Formatter('{"method" : "CRAWLING", "time" : "%(asctime)s", "level" : "%(levelname)s", "message" : "%(message)s"}')
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

sqs = boto3.resource('sqs')
try:
    queue = sqs.get_queue_by_name(QueueName='nvmids')
except:
    queue = sqs.create_queue(QueueName='nvmids',Attributes=
                            {
                                "MaximumMessageSize":"4096",
                                "VisibilityTimeout":"10",
                            })

#### SQS 관련된 Parameter

| 항목  | 의미  |
|----|----|
| DefaultVisibleTimeout | SQS에서 메시지는 특정 Component에 전달된 뒤, 자동으로 삭제 되지 않는데, 그 때문에 중복된 메시지를 전달받을 수 있음. 그래서 한번 전달된 메시지는 visible timeout에 설정된 일정시간 동안은 다시 전달되지 않도록 하고 있음. 이러한 메시지들의 상태를 **inflight**라고 표현함.|
| MessageRetentionPeriod | 메시지 생명주기 / 1분부터 최대 14일까지 지정할 수 있음 |
| Maximum Message Size | 메시지 최대 크기 최대 256Kbytes까지 가능 |
| Delivery Delay | 새로운 메시지가 전달되는 초기 지연 시간. 0초~900초(15분)까지 설정 가능. |
| Receive Message Wait Time | Long Polling을 활성화 함, 0~20초까지 가능 |


## 0. Redis에 item 정보 담기

In [ ]:
redis = Redis(host='localhost',port=6379)
item_df = pd.read_csv("data/sample_item.csv",sep="▒",dtype=str,engine='python')

In [ ]:
for _, row in item_df.iterrows():
    # set of item data about nvmid
    redis.hset("item",row.nv_mid,json.dumps(row.to_dict()))

## 1. SQS(simple Queue Service)

#### SQS에 메시지 보내기

In [ ]:
# nvmid의 집합
nvmids = [key.decode('utf-8') for key in redis.hkeys('item')]

In [ ]:
# nvmids에 보내는 메시지 형식
nvmid_message = lambda nvmid, item, page : {
    "Id" : nvmid,
    "MessageBody": item,
    "MessageAttributes" : {
        "Page" : {
            "StringValue" : page,
            "DataType" : "Number"
        }
    }
}

In [ ]:
# nvmid queue에 Bulk Insert하기 
Entries = []
start = time.time()
for nvmid in nvmids:
    item = redis.hget("item",nvmid).decode('utf-8')
    Entries.append(nvmid_message(nvmid,item,"1"))
    if len(Entries) == 10:
        queue.send_messages(Entries=Entries)
        Entries=[]
if len(Entries) > 0:
    queue.send_messages(Entries=Entries)
    Entries=[]
end = time.time()

#### SQS에서 메시지 받아오기

In [27]:
messages = queue.receive_messages(AttributeNames=["All"],
                                MessageAttributeNames=['Page'],
                                MaxNumberOfMessages=1,
                                WaitTimeSeconds=3)
if len(messages)>0:
    msg = messages[0]    
    if msg.message_attributes is not None:
        item_row = json.loads(msg.body)
        page = msg.message_attributes.get('Page').get("StringValue","1")
    else:
        page = "1"
    msg.delete()

## 2. Crawl and analyze data

In [22]:
base_url = "https://search.shopping.naver.com/detail/review_list.nhn"
headers = {
    "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "accept-encoding":"gzip, deflate, br",
    "accept-language":"ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "user-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
}
params = {
   "nvMid":10010250511,
    "page": 1,
    "reviewSort": "registration",
    "reviewType": "all"
}

In [28]:
params['nvMid'] = item_row['nv_mid']
params['page'] = page

res = requests.post(base_url,params=params,headers=headers,timeout=5)

In [34]:
# 1. 고유 명사 수 세기 (얼마나 단어에 의미 있는 것들이 존재하는가에 대한 지표)
twitter = Twitter()
count_unique_nouns = lambda text : len(set(twitter.nouns(text)))

# 2. 리뷰의 고유 ID 지정해주기
convert_hash = lambda text : sha224(text.encode("utf-8")).hexdigest()

# 3. 리뷰 별 Tag값 지정하기
score_df = pd.read_csv("data/score.csv",sep="▒",engine='python')
def calculate_tag(threshold,review):
    # threshold : tag score의 mimimum value
    # 리뷰 내 토큰 집합을 가져옴
    tokens = set(twitter.morphs(review,stem=True,norm=True))
    # score가 있는 토큰들을 모은 후, type별 score을 매김
    results = score_df.loc[score_df.token.isin(tokens),['score','type']].groupby('type').sum()
    # index
    return results[results.score >=threshold].index.values

In [52]:
bsObj = BeautifulSoup(res.text,'html.parser')

if len(bsObj.find_all("div",{'class':'atc_area'})) == 0 :
    logger.info('no review ... url {}'.format(res.url))
rows = []    
for atc_area in bsObj.find_all("div",{'class':'atc_area'}):
    row = item_row.copy()
    try:
        title_expr =    atc_area.p.text
        row["review_title"] = re.sub("[\n\t]","",title_expr).strip()
    except:
        logger.warning("info is missing... url : {}".format(res_url))
        row['review_title'] = ""
    try:
        atc_expr = atc_area.find("div",{'class':'atc'}).text
        row["review_atc"] = re.sub("[\n\t]","",atc_expr).strip()
    except:
        logger.error('article is missing... url : {}'.format(res_url))
        continue
    try:
        row["review_grade"] = atc_area.find("span",{'class':'curr_avg'}).text
    except:
        logger.warning('grade is missing... url : {}'.format(res_url))
        row['review_grade'] = "0"
    try:
        date_expr = atc_area.find("span",{'class':'date'}).text
        row["review_date"] = re.sub("[^\d.]","",date_expr)
    except:
        logger.warning('date is missing... url : {}'.format(res_url))
        row['review_date'] = datetime.strftime(datetime.now(),format="%Y.%m.%d.")
    try:
        row["review_mall"] = atc_area.find("span",{'class':'path'}).text
    except:
        logger.warning('path is missing... url : {}'.format(res_url))
        row['review_mall'] = ""
        
    row['review_id'] = convert_hash(row["nv_mid"])+\
                convert_hash(row["review_atc"]+row["review_title"]+row['review_date'])
    row['review_accuracy'] = count_unique_nouns(row['review_atc'])
    row['review_tag'] = list(calculate_tag(0.5,row['review_atc']))
    rows.append(row)

## 3. Insert document into ElasticSearch

In [7]:
item_index_name = "item_table"
review_index_name = "review_table"

In [6]:
es = Elasticsearch(host="localhost")
if es.ping():
    # Check Elasticsearch is operating
    print("Elasticsearch is Okay")

Elasticsearch is Okay


In [33]:
def generate_action(_index,_type):
    def _generate_action(_id, _source):
        return {
            "_index"  : _index,
            "_type"   : _type,
            "_id"     : _id,
            "_source" : _source
        }
    return _generate_action

In [32]:
if es.indices.exists(item_index_name):
    print('"items" index exists')
#     if delete_index:
#         es.indices.delete(item_index_name, ignore=[400,404])

"items" index exists
